# Predict Next Purchase

In this example, we build a machine learning application that predicts whether customers will purchase a product within the next shopping period. This application is structured into three important steps:

* Prediction Engineering
* Feature Engineering
* Machine Learning

In the first step, we generate new labels from the data by using [Compose](https://compose.alteryx.com/). In the second step, we generate features for the labels by using [Featuretools](https://docs.featuretools.com/). In the third step, we search for the best machine learning pipeline by using [EvalML](https://evalml.alteryx.com/). After working through these steps, you will learn how to build machine learning applications for real-world problems like predicting consumer spending. Let's get started.

In [ ]:
from demo.predict_next_purchase import load_sample
from matplotlib.pyplot import subplots
import composeml as cp
import featuretools as ft
import evalml

We will use this historical data of online grocery orders provided by Instacart.

In [ ]:
df = load_sample()

df.head()

## Prediction Engineering

> Will customers purchase a product within the next shopping period?

In this prediction problem, we have two parameters:

* The product that a customer can purchase.
* The length of the shopping period.

We can change these parameters to create different prediction problems. For example, will a customer purchase a banana within the next 5 days or an avocado within the next three weeks? These variations can be done by simply tweaking the parameters. This helps us explore different scenarios which is crucial for making better decisions.


### Defining the Labeling Process

Let's start by defining a labeling function that checks if a customer bought a given product. We will make the product a parameter of the function.

In [ ]:
def bought_product(ds, product_name):
    return ds.product_name.str.contains(product_name).any()

### Representing the Prediction Problem

Then, let's represent the prediction problem by creating a label maker with the following parameters:

* The `target_entity` as the columns for the customer ID, since we want to process orders for each customer.
* The `labeling_function` as the function we defined previously.
* The `time_index` as the column for the order time, since shoppings periods are based on the order time.
* The `window_size` as the length of a shopping period. We can easily change this parameter to create variations of the prediction problem.

In [ ]:
lm = cp.LabelMaker(
    target_entity='user_id',
    time_index='order_time',
    labeling_function=bought_product,
    window_size='3d',
)

### Finding the Training Examples

Now, let's run a search to get the training examples by using the following parameters:

* The online grocery orders sorted by the order time.
* The `num_examples_per_instance` to find the number of training examples per customer. In this case, we search for all existing examples.
* The `product_name` as the product to check for purchases. This parameter get passed directly to the our labeling function.
* The `minimum_data` as the minimum amount of data required to build features for the first label. 

In [ ]:
lt = lm.search(
    df.sort_values('order_time'),
    num_examples_per_instance=-1,
    product_name='Banana',
    minimum_data='3d',
    verbose=False,
)

lt.head()

The output from the search is a label times table with three columns:

* The customer ID associated to the orders.
* The start time of the shopping period. This is also known as a cutoff time for building features. Only data that existed beforehand is valid to use for predictions.
* Whether or not the product was purchased in the shopping period. This is calculated by our labeling function.

It can become difficult to track the parameters that were used to create the labels. As a helpful reference, we can look at the label description to understand how the labels were created from the start. The description also shows us the label distribution which we can check for imbalanced labels.

In [ ]:
lt.describe()

We can get a better look at the labels by plotting the distribution and the cumulative count across time.

In [ ]:
%matplotlib inline
fig, ax = subplots(nrows=2, ncols=1, figsize=(6, 8))
lt.plot.distribution(ax=ax[0])
lt.plot.count_by_time(ax=ax[1])
fig.tight_layout(pad=2)

## Feature Engineering

In the previous step, we generated the labels. The next step is to generate the features.

### Representing the Data

We will represent the online grocery orders using an entity set. This way, we can generate features based on the relational structure of the dataset. We currently have a single table of orders where one customer can many orders. This one-to-many relationship can be represented in an entity set by normalizing an entity for the customers. The same can be done for departments, aisles, and products. Let's structure the entity set.

In [ ]:
es = ft.EntitySet('instacart')

es.entity_from_dataframe(
    dataframe=df.reset_index(),
    entity_id='order_products',
    time_index='order_time',
    index='id',
)

es.normalize_entity(
    base_entity_id='order_products',
    new_entity_id='orders',
    index='order_id',
    additional_variables=['user_id'],
    make_time_index=False,
)

es.normalize_entity(
    base_entity_id='orders',
    new_entity_id='customers',
    index='user_id',
    make_time_index=False,
)

es.normalize_entity(
    base_entity_id='order_products',
    new_entity_id='products',
    index='product_id',
    additional_variables=['aisle_id', 'department_id'],
    make_time_index=False,
)

es.normalize_entity(
    base_entity_id='products',
    new_entity_id='aisles',
    index='aisle_id',
    additional_variables=['department_id'],
    make_time_index=False,
)

es.normalize_entity(
    base_entity_id='aisles',
    new_entity_id='departments',
    index='department_id',
    make_time_index=False,
)

es["order_products"]["department"].interesting_values = ['produce']
es["order_products"]["product_name"].interesting_values = ['Banana']
es.plot()

### Calculating the Features

Now, we can generate features by using a method called Deep Feature Synthesis (DFS). This will automatically build features by stacking and applying mathematical operations called primitives across relationships in an entity set. The more structured an entity set is, the better DFS can leverage the relationships to generate better features. Let’s run DFS using the following parameters:

* The `entity_set` as the entity set we structured previously.
* The `target_entity` as the customers, since we want to generate features for each customer. 
* The `cutoff_time` as the label times that we generated previously.

In [ ]:
fm, fd = ft.dfs(
    entityset=es,
    target_entity='customers',
    cutoff_time=lt,
    cutoff_time_in_index=True,
    include_cutoff_time=False,
    verbose=False,
)

fm.head()


There are two outputs from DFS: a feature matrix and feature definitions. The feature matrix is a table that contains the feature values based on the cutoff times from our labels. Feature definitions are features in a list that can be stored and reused later to calculate the same set of features on future data.

## Machine Learning

In the previous steps, we generated the labels and features. The final step is to build the machine learning pipeline.

### Splitting the Data

We will start by extracting the labels from the feature matrix and splitting the data into a training set and holdout set.

In [ ]:
y = fm.pop('bought_product')
splits = evalml.preprocessing.split_data(fm, y, test_size=0.2, random_state=0)
X_train, X_holdout, y_train, y_holdout = splits

### Finding the Best Model

Then, we run a search on the training set for the best machine learning model.

In [ ]:
automl = evalml.AutoMLSearch(problem_type='binary', objective='f1', random_state=0)
automl.search(X_train, y_train, data_checks='disabled', show_iteration_plot=False)

We can print out the details and steps from the best pipeline found.

In [ ]:
automl.best_pipeline.describe()
automl.best_pipeline.graph()

Now, let's score the model performance by evaluating predictions on the holdout set.

In [ ]:
best_pipeline = automl.best_pipeline.fit(X_train, y_train)
score = best_pipeline.score(X_holdout, y_holdout, objectives=['f1'])
dict(score)